In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D
from keras.preprocessing import image
import keras
import os
import pickle
import pyreadstat
from keras.layers import MaxPooling2D, Flatten, Dense, Activation
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# DataModel 

In [2]:
DataModel = Sequential()
DataModel.add(Conv2D(32, (3, 3), input_shape = (40,100,3), activation = 'relu'))
DataModel.add(MaxPooling2D(pool_size = (2, 2)))

DataModel.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
DataModel.add(MaxPooling2D(pool_size = (2, 2)))

DataModel.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'valid'))
DataModel.add(MaxPooling2D(pool_size = (2, 2)))

DataModel.add(Flatten())
DataModel.add(Dense(units = 128, activation = 'relu'))
DataModel.add(Dense(units = 33, activation = 'softmax'))

DataModel.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

DataModel.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 98, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 49, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 24, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 22, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 11, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4224)              0

## preparing traing and test data 

In [3]:
dataGen=ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2)

testGen=ImageDataGenerator(rescale = 1./255)

dataFlow = dataGen.flow_from_directory(directory = 'Letters',target_size = (40,100),
                                              batch_size = 32, class_mode = 'categorical')
testFlow = testGen.flow_from_directory(directory = 'Letters23',target_size = (40,100),
                                             batch_size = 32,class_mode = 'categorical')

Found 22091 images belonging to 33 classes.
Found 528 images belonging to 33 classes.


In [4]:
callback_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
callback_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')


# making training

In [5]:
history = DataModel.fit(dataFlow,epochs = 1,
                    callbacks=[callback_reduce, callback_stop],validation_data = testFlow,)

691/691 [==============================] - 66s 95ms/step - loss: 1.8546 - accuracy: 0.4941 - val_loss: 0.6897 - val_accuracy: 0.8163


In [ ]:
## saving and testing

In [ ]:
DataModel.save("4poch10model.h5")

In [6]:
def convertChar(response):
    if response[0][0] == 1:
        return('ა')
    elif response[0][1] == 1:
        return ('ბ')
    elif response[0][2] == 1:
        return ('გ')
    elif response[0][3] == 1:
        return ('დ')
    elif response[0][4] == 1:
        return ('ე')
    elif response[0][5] == 1:
        return ('ვ')
    elif response[0][6] == 1:
        return ('ზ')
    elif response[0][7] == 1:
        return ('თ')
    elif response[0][8] == 1:
        return ('ი')
    elif response[0][9] == 1:
        return ('კ')
    elif response[0][10] == 1:
        return ('ლ')
    elif response[0][11] == 1:
        return ('მ')
    elif response[0][12] == 1:
        return ('ნ')
    elif response[0][13] == 1:
        return ('ო')
    elif response[0][14] == 1:
        return ('პ')
    elif response[0][15] == 1:
        return ('ჟ')
    elif response[0][16] == 1:
        return ('რ')
    elif response[0][17] == 1:
        return ('ს')
    elif response[0][18] == 1:
        return ('ტ')
    elif response[0][19] == 1:
        return ('უ')
    elif response[0][20] == 1:
        return ('ფ')
    elif response[0][21] == 1:
        return ('ქ')
    elif response[0][22] == 1:
        return ('ღ')
    elif response[0][23] == 1:
        return ('ყ')
    elif response[0][24] == 1:
        return ('შ')
    elif response[0][25] == 1:
        return ('ჩ')
    elif response[0][26] == 1:
        return ('ც')
    elif response[0][27] == 1:
        return ('ძ')
    elif response[0][28] == 1:
        return ('წ')
    elif response[0][29] == 1:
        return ('ჭ')
    elif response[0][30] == 1:
        return ('ხ')
    elif response[0][31] == 1:
        return ('ჯ')
    elif response[0][32] == 1:
        return ('ჰ')

In [7]:

img1 = image.load_img("Letters/a1/a__!_kolhety_normal.png", target_size = (40,100))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1, axis = 0)
res2 = DataModel.predict(img1)

bigSum = 0
biggod = 0
bigmin = 0

for k in range(33):
    i = k +1
    st = "a"    

    if i < 10:
        st = "a"
    elif i < 20:
        st = "b"
    elif i <30:
        st = "c"
    else:
        st = "d"

    stPath = "Letters23/" + st + str(i) + "/"
    directory = os.fsencode(stPath)
    res2[0][k] = 1
    res = convertChar(res2)
    res2[0][k] = 0

    sum = 0
    god = 0
    min = 0
    for file in os.listdir(directory):
            filename = os.fsdecode(file)
            path = stPath + filename
            img = image.load_img(path, target_size = (40,100))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis = 0)
            result = DataModel.predict(img)
            result = convertChar(result)
            #print(result)
            if(result == res):
                god +=1
                sum +=1
            else:
                min +=1
                sum +=1

    #print(res)
    #print ('sum  : {}'.format(sum))
    #print ('god  : {}'.format(god))
    #print ('min  is: {}'.format(min))
   # print ('percent  is: {}'.format(god/sum))
    bigSum += sum
    biggod += god
    bigmin += min
   # print("---------")
    
print("-##############")
print ('sum  : {}'.format(bigSum))
print ('god  : {}'.format(biggod))
print ('min  is: {}'.format(bigmin))
print ('percent  is: {}'.format(biggod/bigSum))

-##############
sum  : 528
god  : 429
min  is: 99
percent  is: 0.8125
